# Discover SCPs

Discovers all the SCPs currently applied per account in the organization

In [24]:
import pandas as pd
import boto3
import logging
import json
import numpy as np

## Get session

In [25]:
session = boto3.Session()
orgs = session.client('organizations')

## Get account list

In [26]:
accounts = []
paginator = orgs.get_paginator('list_accounts')
for page in paginator.paginate():
    accounts.extend(page['Accounts'])
df = pd.DataFrame(accounts)

## List SCPs

In [27]:
paginator = orgs.get_paginator('list_policies')
options = dict(
    Filter='SERVICE_CONTROL_POLICY'
)
scps = []
for page in paginator.paginate(**options):
    scps.extend(page['Policies'])

In [28]:
## Describe policies

policies = {}
for policy in scps:
    policy_doc = orgs.describe_policy(PolicyId=policy['Id'])
    policies[policy['Id']] = json.loads(policy_doc['Policy']['Content'])

## List targets for policy (SCP)

In [29]:
paginator = orgs.get_paginator('list_targets_for_policy')
targets = []
for policy in scps:
    options = dict(
        PolicyId=policy['Id']
    )
    for page in paginator.paginate(**options):
        targets.append({
            'Id': policy['Id'],
            'Targets': page['Targets']
        })

## Convert to dataframes

In [30]:
scps = pd.DataFrame(scps)

targets = pd.DataFrame(targets)

targets = targets.explode('Targets')

## Join scps and targets

In [31]:
scps = scps.merge(targets, on='Id')

## Extract account id and target type from target

In [32]:
scps = scps.loc[
    scps['Targets'].notna()
].copy()

In [33]:
if scps.empty: raise SystemExit('SCPs disabled or not applied')

In [34]:
def get_target_id(x):
    try:
        return x['TargetId'], x['Type']
    except:
        return

scps[['TargetId', 'TargetType']] = scps['Targets'].apply(get_target_id).apply(pd.Series)

## Map organization

In [35]:
def list_parents(child_id):
    try:
        paginator = orgs.get_paginator('list_parents')
        options = {
            'ChildId': child_id
        }
        results = []
        for page in paginator.paginate(**options):
            results.extend(page['Parents'])
        return results
    except Exception as err:
        return err

In [36]:
scps['Parents'] = scps['TargetId'].apply(list_parents)

scps = scps['Parents'].explode().apply(pd.Series).rename(columns={
    'Id': 'ParentId',
    'Type': 'ParentType'
}).drop(columns=[0]).join(scps)

## Clean up SCPS not applied

In [37]:
scps.dropna(subset=['TargetId'], inplace=True)

## Build scp for each account

In [38]:
def consolidate_path(target_id, dataframe):
    """Given a target and a dataframe of accounts and SCPs, traverse the account to the root and return a list of hops."""    
    hops = [target_id]
    
    try:
        parent = dataframe.loc[
            dataframe['TargetId'] == target_id, 'ParentId'
        ].values[0]
    except Exception as err:
        logger.error(f'Unable to traverse node: {err}')
        return 
    
    if parent is not np.NaN:
        hops.extend(consolidate_path(parent, dataframe))
    return hops

In [39]:
df['Path'] = df['Id'].apply(consolidate_path, dataframe=scps)

In [40]:
def get_scps(targets, scps, policies):
    """Return policies for given target list."""
    policy_statements = []
    
    if not targets: return
    for target in targets:
        policy_ids = scps.loc[
            scps['TargetId'] == target
        ]['Id'].values
        for pid in policy_ids:
            policy_statements.extend(policies[pid]['Statement'])
        
    return policy_statements

In [43]:
df['Scps'] = df['Path'].apply(get_scps, scps=scps, policies=policies)

df_scps = df['Scps'].explode().apply(pd.Series).join(df).astype(str).drop_duplicates().drop(columns='Scps')

In [20]:
## Save dataset
df_scps.to_csv(OutputFile, index=False)

NameError: name 'OutputFile' is not defined